# 원불교 교전 데이터 대상 토크나이저 훈련

## 1. 개요

원불교 교전 데이터를 대상으로 토크나이저를 훈련합니다.

* 데이터셋:
  * 원불교 교전
* 토크나이저:
  * 허깅 페이스 [Tokenizers](https://huggingface.co/docs/tokenizers/index) 라이브러리의 [`Tokenizer`](https://huggingface.co/docs/tokenizers/api/tokenizer) 클래스

## 2. 말뭉치 준비

In [1]:
corpus_file = "./won/data/won04-gyojeon.txt"

말뭉치 파일에 대하여 아래 정보를 출력합니다.

* 총 줄 수
* 총 단어 수(단어를 공백으로 구분할 때)
* 가장 많은 단어를 포함하고 있는 줄의 단어 수
* 가장 많은 단어를 포함하고 있는 줄의 텍스트

In [2]:
def count_lines_and_words(filepath):
    try:
        with open(filepath, 'r', encoding='utf-8') as file:
            lines = file.readlines()
            line_count = len(lines)
            word_count = 0
            max_line_length = 0
            max_line = ""
            for line in lines:
                words = line.split() # 공백으로 단어 분리
                line_length = len(words)
                if line_length > max_line_length:
                    max_line_length = line_length
                    max_line = line
                word_count += line_length
            return line_count, word_count, max_line_length, max_line
    except FileNotFoundError:
        print(f"오류: 파일 '{filepath}'를 찾을 수 없습니다.")
        return None, None, None, None
    except Exception as e:
        print(f"파일을 읽는 중 오류가 발생했습니다: {e}")
        return None, None, None, None

# 함수 호출 및 결과 출력
lines, words, max_line_length, max_line = count_lines_and_words(corpus_file)

if lines is not None and words is not None:
    print(f"파일 분석 결과:")
    print(f"  * 총 줄 수  : {lines:8,}")
    print(f"  * 총 단어 수: {words:8,}")
    print("  * 가장 많은 단어를 포함하고 있는 줄:")
    print(f"    - 단어 수: {max_line_length:8,}")
    print(f"    - 텍스트 : {max_line}")

파일 분석 결과:
  * 총 줄 수  :    1,082
  * 총 단어 수:   59,541
  * 가장 많은 단어를 포함하고 있는 줄:
    - 단어 수:      362
    - 텍스트 : 17.한 사람이 대종사께 뵈옵고 여러 가지로 담화하는 가운데 [전주․이리 사이의 경편철도(輕便鐵道)는 본래 전라도 각지의 부호들이 주식 출자로 경영하는 것이라, 그들은 언제나 그 경편차를 무료로 이용하고 다닌다.] 하면서 매우 부러워하는 태도를 보이거늘, 대종사 말씀하시기를 [그대는 참으로 가난하도다. 아직 그 차 하나를 그대의 소유로 삼지 못하였는가.] 그 사람이 놀라 여쭙기를 [경편차 하나를 소유하자면 상당한 돈이 있어야 할 것이온데 이 같은 무산자로서 어떻게 그것을 소유할 수 있사오리까.] 대종사 말씀하시기를 [그러므로, 그대를 가난한 사람이라 하였으며, 설사 그대가 경편차 하나를 소유하였다 할지라도 나는 그것으로 그대를 부유한 사람이라고는 아니할 것이니, 이제 나의 살림하는 이야기를 좀 들어보라. 나는 저 전주 경편차뿐 아니라 나라 안의 차와 세계의 모든 차까지도 다 내 것을 삼은지가 벌써 오래 되었노니, 그대는 이 소식을 아직도 모르는가.] 그 사람이 더욱 놀라 사뢰기를 [그 말씀은 실로 요량 밖의 교훈이시므로 어리석은 소견으로는 그 뜻을 살피지 못하겠나이다.] 대종사 말씀하시기를 [사람이 기차 하나를 자기의 소유로 하려면 거액(巨額)의 자금이 일시에 들어야 할 것이요, 운영하는 모든 책임을 직접 담당하여 많은 괴로움을 받아야 할 것이나, 나의 소유하는 법은 그와 달라서 단번에 거액을 들이지도 아니하며, 모든 운영의 책임을 직접 지지도 아니하고, 다만 어디를 가게 되면 그 때마다 얼마씩의 요금만 지불하고 나의 마음대로 이용하는 것이니, 주야로 쉬지 않고 우리 차를 운전하며, 우리 철도를 수선하며, 우리 사무를 관리하여 주는 모든 우리 일꾼들의 급료와 비용이 너무 싸지 아니한가. 또, 나는 저번에 서울에 가서 한양 공원에 올라가 산책하면서 맑은 공기를 한 없

In [3]:
sample_text_for_test = "물질이 개벽되니 정신을 개벽하자"

sample_texts_for_test = [
    "물질이 개벽되니 정신을 개벽하자",
    "응용(應用)하는 데 온전한 생각으로 취사하기를 주의할 것이요",
    "19.대종사 말씀하시기를 [스승이 법을 새로 내는 일이나"
]

## 3. 토크나이저 훈련

알고리즘과 단어 개수를 아래와 같이 변경해 가면서 훈련하고 각각의 조합으로 디렉토리 <*알고리즘*>_<*단어 개수*>를 만들어 훈련 결과를 저장합니다.

* 알고리즘: BPE, Unigram, WordPiece
* 단어 개수: 4000, 5000, 7000, 10000

In [4]:
from tokenizers import Tokenizer
from tokenizers.models import BPE, Unigram, WordPiece
from tokenizers.trainers import BpeTrainer, UnigramTrainer, WordPieceTrainer
from tokenizers.pre_tokenizers import Whitespace

def create_tokenizer_and_trainer(model_name, vocab_size):
    if model_name == "bpe":
        tokenizer = Tokenizer(BPE(unk_token="[UNK]"))
        trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                            vocab_size=vocab_size,
                            limit_alphabet=limit_alphabet,
                            min_frequency=min_frequency,
                            show_progress=False)
    elif model_name == "unigram":
        tokenizer = Tokenizer(Unigram())
        trainer = UnigramTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                            unk_token="[UNK]",
                            vocab_size=vocab_size,
                            show_progress=False)
    elif model_name == "wordpiece":
        tokenizer = Tokenizer(WordPiece(unk_token="[UNK]"))
        trainer = WordPieceTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"],
                            vocab_size=vocab_size,
                            limit_alphabet=limit_alphabet,
                            min_frequency=min_frequency,
                            show_progress=False)

    tokenizer.pre_tokenizer = Whitespace()

    return tokenizer, trainer

In [5]:
import os

def train_tokenizer(
    model_names,
    vocab_sizes,
    limit_alphabet,
    min_frequency,
    data_files,
    output_dir
):
    for model_name in model_names:
        for vocab_size in vocab_sizes:
            # 토크나이저와 트레이너 생성
            tokenizer, trainer = create_tokenizer_and_trainer(model_name, vocab_size)
            # 훈련
            tokenizer.train(data_files, trainer)
            # 저장
            tokenizer_dir = os.path.join(output_dir, f"{model_name}_{vocab_size}")
            if not os.path.exists(tokenizer_dir):
                os.makedirs(tokenizer_dir, exist_ok=True)
            tokenizer_file = os.path.join(tokenizer_dir, "tokenizer.json")
            tokenizer.save(tokenizer_file)
            print(f"Tokenizer file: {tokenizer_file}")

In [6]:
data_files = [
    corpus_file
]

output_dir = "./won/tokenizers"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model_names = ["bpe", "unigram", "wordpiece"]
vocab_sizes = [2000, 3000, 4000, 5000, 7000, 10000]
limit_alphabet = 6000
min_frequency = 5

train_tokenizer(model_names, vocab_sizes, limit_alphabet, min_frequency, data_files, output_dir)

Tokenizer file: ./won/tokenizers/bpe_2000/tokenizer.json
Tokenizer file: ./won/tokenizers/bpe_3000/tokenizer.json
Tokenizer file: ./won/tokenizers/bpe_4000/tokenizer.json
Tokenizer file: ./won/tokenizers/bpe_5000/tokenizer.json
Tokenizer file: ./won/tokenizers/bpe_7000/tokenizer.json
Tokenizer file: ./won/tokenizers/bpe_10000/tokenizer.json
Tokenizer file: ./won/tokenizers/unigram_2000/tokenizer.json
Tokenizer file: ./won/tokenizers/unigram_3000/tokenizer.json
Tokenizer file: ./won/tokenizers/unigram_4000/tokenizer.json
Tokenizer file: ./won/tokenizers/unigram_5000/tokenizer.json
Tokenizer file: ./won/tokenizers/unigram_7000/tokenizer.json
Tokenizer file: ./won/tokenizers/unigram_10000/tokenizer.json
Tokenizer file: ./won/tokenizers/wordpiece_2000/tokenizer.json
Tokenizer file: ./won/tokenizers/wordpiece_3000/tokenizer.json
Tokenizer file: ./won/tokenizers/wordpiece_4000/tokenizer.json
Tokenizer file: ./won/tokenizers/wordpiece_5000/tokenizer.json
Tokenizer file: ./won/tokenizers/wordp

## 4. 토크나이저 사용

In [7]:
import sys, traceback
from transformers import DistilBertTokenizerFast, PreTrainedTokenizerFast

def tokenize_and_print(model_names, vocab_sizes, texts):
    for text in texts:
        print("#" * 80)
        print("TEXT: '" + text + "'")
        for model_name in model_names:
            print(f">>> {model_name}")
            for vocab_size in vocab_sizes:
                try:
                    tokenizer_file = os.path.join(output_dir, f"{model_name}_{vocab_size}", "tokenizer.json")
                    tokenizer = Tokenizer.from_file(tokenizer_file)
                    print(f"{vocab_size:5}: {tokenizer.encode(text).tokens}")
                except Exception as e:
                    print(f"{vocab_size:5}: FAIL - {e}")
                    traceback.print_exc(file=sys.stdout)

In [8]:
tokenize_and_print(model_names, vocab_sizes, sample_texts_for_test)

################################################################################
TEXT: '물질이 개벽되니 정신을 개벽하자'
>>> bpe
 2000: ['물', '질', '이', '개', '벽', '되', '니', '정신', '을', '개', '벽', '하', '자']
 3000: ['물질', '이', '개', '벽', '되', '니', '정신을', '개', '벽', '하', '자']
 4000: ['물질', '이', '개', '벽', '되니', '정신을', '개', '벽', '하', '자']
 5000: ['물질', '이', '개벽', '되니', '정신을', '개벽', '하자']
 7000: ['물질', '이', '개벽', '되니', '정신을', '개벽', '하자']
10000: ['물질', '이', '개벽', '되니', '정신을', '개벽', '하자']
>>> unigram
 2000: ['물질', '이', '개', '벽', '되', '니', '정신', '을', '개', '벽', '하자']
 3000: ['물질', '이', '개', '벽', '되', '니', '정신', '을', '개', '벽', '하자']
 4000: ['물질', '이', '개벽', '되', '니', '정신', '을', '개벽', '하자']
 5000: ['물질', '이', '개벽', '되니', '정신', '을', '개벽', '하자']
 7000: ['물질이', '개벽', '되니', '정신', '을', '개벽', '하자']
10000: ['물질이', '개벽', '되니', '정신', '을', '개벽', '하자']
>>> wordpiece
 2000: ['물', '##질', '##이', '개', '##벽', '##되', '##니', '정', '##신', '##을', '개', '##벽', '##하', '##자']
 3000: ['물', '##질', '##이', '개', '##벽', '##되', '##니', '정', '##신', 